## Convolutional_NN

In [42]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size(filter), stride, padding=0, dilation=1, groups=1, bias = True)
        # 1 input image channel, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Sequential(
                            nn.Conv2d(1,6,5),
                            nn.ReLU()
                            )
        
        # out_channels = 6이므로 in_channels = 6
        # out_channels = 16이고 kernal_size = 5로 해보겠습니다.
        
        self.conv2 = nn.Sequential(
                        nn.Conv2d(6,16,5),
                        nn.ReLU()
                        )
        
        # torch.nn.MaxPool2d(kernel_size, stride = None,padding = 0, dilation = 1, return_indices=False, ceil_mode = False)
        self.maxpool= nn.MaxPool2d(kernel_size=2, stride = 2)
        
        # an affine operation: y = Wx+b (어파인 연산)
        # typical cnn에서는 conv layer뒤에 fully connected layer를 붙입니다.
        self.fc1 = nn.Sequential(
                        nn.Linear(16*5*5,120),
                        nn.ReLU()
                        )
        self.fc2 = nn.Sequential(
                        nn.Linear(120,84),
                        nn.ReLU()
                        )
        self.fc3 = nn.Sequential(
                        nn.Linear(84,10),
                        nn.ReLU()
                        )
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.maxpool(x)
          
        x = self.conv2(x)
        x = self.maxpool(x)
            
        # -1은 몇개가 들어오는지 모른다는 뜩
        #  num_flat_features 는 1X? 형태로 flat하게 reshape합니다.
            
        x = x.view(-1, self.num_flat_features(x))
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
            
        return x
        
    # x.size()[0]은 input 갯수이기 때문에 제외합니다. 각각의 input값들을 1X>형태로 reshape합니다.
    # torch.randn(5,3,3)을 생각해보면 개수를 의미하는 5를 제외하고 3X3=9의 값을 return합니다.
        
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
net = Net()
print(net)        

Net(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): ReLU()
  )
  (fc3): Sequential(
    (0): Linear(in_features=84, out_features=10, bias=True)
    (1): ReLU()
  )
)


- forward 이후에 backward는 autograd를 이용해 자동으로 정의할 수 있습니다.
- net의 weight들은 net.parameters()에 의해 return됩니다.

In [43]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # 5X5 행렬 6개

10
torch.Size([6, 1, 5, 5])


In [44]:
temp_input = torch.randn(1,1,32,32) # 32X32 하나를 input으로 넣어줍니다.

In [45]:
net.forward(temp_input)


tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0017, 0.0000, 0.1069, 0.0000, 0.0000,
         0.0000]], grad_fn=<ThresholdBackward0>)

input이 왜 1X1X32X32로 들어가야할까요? torch.nn에서는 mini-batch만 지원합니다. 따라서 nnconv2d는 nsamples X nChannels X Height X Width의 4차원 tensor를 입력으로 합니다. 1개의 값을 input channel(1)에 넣으니 1X1이 됩니다

In [ ]:
# 각 연산의 gradient가 저장되어 있는데 그 값을 초기화 해줍니다.
net.zero_grad()
out.backward(torch.randn(1,10)) # loss를 정의하지 않았으므로 임의의 값으로 back prop을 합니다.

torch.tensor는 autograd 연산을 지원합니다. 또한 tensor의 gradient를 갖고 있습니다. back prop할때마다 이를 초기화 해야합니다.
nn.module은 weight를 캡슐화해서 GPU로 이동, 내보내기 불러오기등의 작업을 도와줍니다.
nn.parameter는 tensor의 종류로 module에 할당될 때 자동으로 weight로 등록됩니다.

## Loss function(손실함수)

손실함수는 output, target을 한 쌍의 입력으로 받아 output이 target으로부터 얼마나 떨어져 있는지를 추정하는 값을 계산합니다. output은 net이 계산한 추정값(출력), target은 실제 값(정답)입니다. 

In [56]:
output = net(input)
target = torch.arange(1,11,dtype = torch.float)
# torch.unsqueeze(target, dim=0)과 같습니다. reshape가 목적
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(38.2611, grad_fn=<MseLossBackward>)


In [57]:
# grad_fn.next_functions[]을 통해 gradient function을 추적할 수 있습니다.
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0])

In [58]:
net.zero_grad()

print('conv1.bias.grad before backward')

# nn.Sequential로 여러 function을 묶어뒀을 경우 []를 통해 순서대로 출력할 수 있습니다.
print(net.conv1[0].bias.grad)
# 값이 모두 0으로 초기화되어 있습니다.

loss.backward() # gradient를 계산

print('conv1.bias.grad after backward')
print(net.conv1[0].bias.grad)
print(net.conv2[0].bias.grad)
print(net.fc1[0].bias.grad)
print(net.fc2[0].bias.grad)
print(net.fc3[0].bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0562,  0.0298,  0.0420,  0.0115,  0.0725,  0.0303])
tensor([ 0.0000, -0.0685,  0.1242,  0.1937,  0.0541,  0.0095,  0.0256, -0.0544,
         0.0030, -0.1352, -0.0774, -0.0222,  0.0410,  0.1506, -0.0082, -0.0541])
tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.5945e-02,  9.1963e-02,
         0.0000e+00, -4.3009e-02,  0.0000e+00,  0.0000e+00,  2.4676e-02,
         0.0000e+00, -4.6041e-02,  7.4650e-06,  2.5243e-02, -4.7554e-02,
        -8.7562e-02,  0.0000e+00,  6.9426e-02,  0.0000e+00,  0.0000e+00,
        -2.2287e-02, -5.1680e-02, -4.9050e-02, -9.6443e-02,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  1.1422e-01, -6.0631e-02, -8.9633e-02,
         0.0000e+00, -1.1717e-01, -2.8079e-02,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  2.3731e-02,  5.8281e-02,  9.3793e-04,
         3.6191e-02, -1.0880e-03, -2.6754e-02,  0.0000e+00,  8.6505e-02,
         9.5071e-02,  0

## Weight update(가중치 갱신)

In [60]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr = 0.01) # Adam optimizer를 사용, learning rate 는 0.01입니다.

# 학습과정(가중치 갱신 과정)은 다음과 같다
optimizer.zero_grad() # 기존의 변화도에 대해 누적되는 것을 막기 위해 zero_grad()로 초기화
output = net(temp_input)
loss = criterion(output, target)
loss = criterion(output, target)
loss.backward()
optimizer.step()